## Введение в обработку естественного языка
### Урок 2. Создание признакового пространства
### Практическое задание

**Продолжим обработку данных с Твиттера.** 

In [1]:
from IPython.core.display import display, HTML, Image
display(HTML("<style>.container { width:77% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)
import os
import time

In [3]:
plt.rcParams.update({'font.size': 14})
pd.set_option('precision', 3)
pd.set_option('max_columns', 100)
pd.set_option('display.float_format', lambda x: '%.5f' % x)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.set_option('max_colwidth', 300)

In [4]:
# Загрузка из файла
combine_df = pd.read_pickle('../lesson01/combine_df.pkl')
# Сохранение в файл
# combine_df.to_pickle('../lesson01/combine_df.pkl')

In [5]:
combine_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49159 entries, 0 to 49158
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          49159 non-null  int64  
 1   label                       31962 non-null  float64
 2   tweet                       49159 non-null  object 
 3   clean_tweet                 49159 non-null  object 
 4   clean_tweet_token           49159 non-null  object 
 5   clean_tweet_token_filtered  49159 non-null  object 
 6   clean_tweet_stemmed         49159 non-null  object 
 7   clean_tweet_lemmatized      49159 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 3.0+ MB


In [6]:
combine_df.head()

,id,label,tweet,clean_tweet,clean_tweet_token,clean_tweet_token_filtered,clean_tweet_stemmed,clean_tweet_lemmatized
0,1,0.00000,@user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run,when father is dysfunctional and is so selfish he drags his kids into his dysfunction run,"[when, father, is, dysfunctional, and, is, so, selfish, he, drags, his, kids, into, his, dysfunction, run]","[father, dysfunctional, selfish, drags, kids, dysfunction, run]","[father, dysfunct, selfish, drag, kid, dysfunct, run]","[father, dysfunctional, selfish, drag, kid, dysfunction, run]"
1,2,0.00000,@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked,thanks for lyft credit cannot use cause they do not offer wheelchair vans in pdx disapointed getthanked,"[thanks, for, lyft, credit, can, not, use, cause, they, do, not, offer, wheelchair, vans, in, pdx, disapointed, getthanked]","[thanks, lyft, credit, use, cause, offer, wheelchair, vans, pdx, disapointed, getthanked]","[thank, lyft, credit, use, caus, offer, wheelchair, van, pdx, disapoint, getthank]","[thank, lyft, credit, use, cause, offer, wheelchair, vans, pdx, disapointed, getthanked]"
2,3,0.00000,bihday your majesty,bihday your majesty,"[bihday, your, majesty]","[bihday, majesty]","[bihday, majesti]","[bihday, majesty]"
3,4,0.00000,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦,model love you take with you all the time in ur,"[model, love, you, take, with, you, all, the, time, in, ur]","[model, love, take, time, ur]","[model, love, take, time, ur]","[model, love, take, time, ur]"
4,5,0.00000,factsguide: society now #motivation,factsguide society now motivation,"[factsguide, society, now, motivation]","[factsguide, society, motivation]","[factsguid, societi, motiv]","[factsguide, society, motivation]"


In [8]:
clean_tweet_stemmed = np.array([' '.join(x) for x in combine_df['clean_tweet_stemmed']])

In [10]:
clean_tweet_lemmatized = np.array([' '.join(x) for x in combine_df['clean_tweet_lemmatized']])

**1. Создайте мешок слов с помощью sklearn.feature_extraction.text.CountVectorizer.fit_transform(). Применим его к 'tweet_stemmed' и 'tweet_lemmatized' отдельно.**  
<li>Игнорируем слова, частота которых в документе строго превышает порог 0.9 с помощью max_df</li>
<li>Ограничим количество слов, попадающий в мешок, с помощью max_features = 1000</li>
<li>Исключим стоп-слова с помощью stop_words='english'</li>
<li>Отобразим Bag-of-Words модель как DataFrame. columns необходимо извлечь с помощью CountVectorizer.get_feature_names()</li>

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer(max_df = 0.9, max_features = 1000, stop_words='english')

#### CountVectorizer - clean_tweet_stemmed

In [9]:
# Создаем the Bag-of-Words модель колонки clean_tweet_stemmed
bag_of_words = count_vectorizer.fit_transform(clean_tweet_stemmed)

# Отобразим Bag-of-Words модель как DataFrame
feature_names = count_vectorizer.get_feature_names()
CountVectorizer_cts = pd.DataFrame(bag_of_words.toarray(), columns = feature_names)
CountVectorizer_cts.head(5)

,abl,absolut,accept,account,act,action,actor,actual,ad,adapt,add,adventur,affirm,afternoon,age,ago,agre,ahead,aist,album,aliv,allahsoil,allow,alon,alreadi,altwaystoh,alway,amaz,america,american,amp,angel,anger,angri,anim,anniversari,announc,anoth,answer,anti,anxieti,anymor,anyon,anyth,app,appl,appreci,arriv,ask,ass,attack,august,avail,award,away,awesom,aww,babi,bad,bag,balanc,ball,bar,bc,bday,beach,bear,beat,beauti,becom,bed,beer,begin,believ,benefit,best,bestfriend,besti,better,big,bigot,bihday,bike,bing,bird,bit,bitch,black,blame,bless,blm,block,blog,blogger,blond,blue,blur,board,bodi,bong,book,bore,bought,box,boy,boyfriend,brand,break,breakfast,brexit,bride,bring,bro,broken,brother,brown,buffalo,build,bull,busi,buy,cake,calm,came,camp,campaign,cantwait,car,card,care,case,cat,caus,celebr,challeng,chanc,chang,chase,check,cheer,child,children,chill,choic,choos,christian,christina,christma,church,citi,claim,class,clean,client,climb,close,cloth,club,coach,coffe,cold,collect,colleg,color,colour,come,comment,commun,compani,complet,conc,confer,confus,congrat,congratul,connect,content,continu,control,cook,cool,count,countdown,countri,coupl,cours,cover,crazi,creat,creativ,cri,cultur,cup,current,custom,cut,cute,da,dad,daddi,daili,damn,danc,dark,date,daughter,day,dead,deal,dear,death,decid,deep,definit,delet,deletetweet,depress,deserv,design,despit,develop,die,diet,differ,dinner,direct,disappoint,disgust,disney,dj,dog,domin,donald,dont,draw,dream,dress,drink,drive,drop,dude,eah,earli,easi,eat,edit,educ,elect,els,em,...,seen,self,selfi,sell,send,sens,seri,servic,session,set,sex,sexi,shall,shame,share,shi,shit,sho,shock,shoe,shoot,shop,shot,sick,sign,silver,simpl,simul,sinc,sing,singl,sister,sit,sjw,sky,sleep,slut,small,smh,smile,snapchat,snapshot,social,someon,someth,sometim,son,song,soon,sorri,soul,sound,sourc,space,speak,special,speech,spend,spo,spring,st,sta,staff,stage,stand,star,state,stay,step,stomp,stop,store,stori,stream,street,stress,strong,stuck,student,studi,studio,stuff,stupid,style,success,suck,suicid,summer,sun,sunday,sunni,sunset,sunshin,super,suppo,suppoer,sure,surpris,surviv,sweet,swim,ta,tag,talent,talk,target,task,tattoo,tbt,tea,teach,teacher,team,tear,tech,teen,tell,term,terribl,terror,terrorist,test,tgif,th,thank,thankyou,thing,think,thought,thursday,ticket,till,time,tip,tire,today,togeth,told,tomorrow,tonight,took,total,touch,tour,town,tragedi,tragic,train,travel,treat,trend,tri,trip,true,truli,trump,trust,truth,tuesday,tune,turn,tv,tweet,twitter,type,udtapunjab,uk,understand,unit,univers,updat,upset,ur,usa,usd,use,vacat,vast,vega,vegan,vibe,vicin,victim,video,view,vine,violenc,visit,voic,vote,vs,wait,wake,walk,wan,want,war,wast,watch,water,way,weak,wear,weather,web,websit,wed,wednesday,week,weekend,welcom,went,wet,whatev,white,wife,wild,win,wine,winner,wish,woh,woman,women,wonder,word,work,workout,world,worri,worst,wow,write,wrong,wtf,xx,xxx,ya,yay,ye,yeah,year,yesterday,yo,yoga,york,young,youtub,yr,yummi
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,

#### CountVectorizer - clean_tweet_lemmatized

In [11]:
# Создаем the Bag-of-Words модель колонки clean_tweet_lemmatized
bag_of_words = count_vectorizer.fit_transform(clean_tweet_lemmatized)

# Отобразим Bag-of-Words модель как DataFrame
feature_names = count_vectorizer.get_feature_names()
CountVectorizer_ctl = pd.DataFrame(bag_of_words.toarray(), columns = feature_names)
CountVectorizer_ctl.head(5)

,able,absolutely,accept,account,act,action,actor,actually,adapt,add,adventure,affirmation,affirmations,afternoon,age,ago,agree,ahead,aist,album,alive,allahsoil,alligator,allow,altwaystoheal,amaze,america,american,americans,amp,anger,angry,animals,anniversary,announce,answer,anti,anxiety,anymore,app,apple,appreciate,arrive,ask,ass,attack,august,available,award,away,awesome,aww,baby,bad,bag,balance,ball,bank,bar,bc,bday,beach,bear,beat,beautiful,beauty,bed,beer,begin,behappy,believe,benefit,best,better,big,bihday,bike,bing,bird,bitch,bite,black,blame,bless,blm,block,blog,blogger,blonde,blue,blur,board,body,bong,book,bore,box,boy,boyfriend,boys,brand,break,breakfast,brexit,bride,bring,bro,broker,brother,brown,buffalo,build,bull,business,busy,buy,cake,calm,camp,campaign,cantwait,car,card,care,case,cat,catch,cause,celebrate,celebration,challenge,chance,change,chase,check,cheer,child,children,chill,chocolate,choice,choose,christina,christmas,church,city,claim,class,clean,click,climb,close,clothe,club,coach,cock,coffee,cold,college,color,come,comedy,comment,community,company,complete,conce,conference,confuse,congrats,congratulations,content,continue,control,cook,cool,count,countdown,country,couple,course,cover,crazy,create,creative,culture,cup,customer,customers,cut,cute,da,dad,daddy,dads,daily,damn,dance,dark,date,daughter,day,days,dead,deal,dear,death,decide,deep,delete,deletetweets,delicious,depress,depression,deserve,design,despite,development,die,diet,different,dinner,direct,disappoint,disgust,disney,dj,dog,dominate,donald,dont,download,draw,dream,dress,drink,drive,drop,eah,early,easy,eat,education,election,em,email,emotional,emotions,end,energy,...,shame,share,shit,sho,shock,shoe,shoot,shop,shy,sick,sign,silver,simple,simulation,simulator,sing,single,sister,sit,sjw,sky,sleep,slut,small,smh,smile,snapchat,snapshot,social,son,song,songs,soon,sorry,soul,sound,source,south,space,spain,speak,special,spend,spos,spring,squad,st,sta,staed,staff,stage,staing,stand,star,stas,state,stay,step,stick,stomp,stop,store,stories,story,stream,street,stress,strong,students,studio,study,stuff,stupid,style,success,successful,suck,summer,sun,sunday,sunny,sunset,sunshine,super,suppo,suppoers,sure,surprise,survive,sweet,swim,ta,tag,talk,tampa,target,task,tattoo,tbt,tea,teach,team,tear,tech,teen,tell,term,terrorism,test,texas,tgif,th,thank,thankful,thankyou,thing,things,think,tho,thoughts,throw,thursday,ticket,till,time,tip,tire,today,tomorrow,tonight,totally,touch,tour,town,track,tragedy,tragic,train,travel,treat,tree,trend,trip,true,truly,trump,trust,truth,try,tuesday,tune,turn,tv,tweet,twitter,type,udtapunjab,uk,understand,unite,university,update,upset,ur,usa,usd,use,useful,vacation,vast,vegan,vegas,vibes,vicinity,victims,video,videos,view,vine,violence,visit,voice,vote,vs,wait,wake,wakeup,walk,wan,want,war,waste,watch,water,way,ways,weak,wear,weather,web,website,wed,wednesday,week,weekend,weeks,welcome,wet,white,wife,wild,win,wine,winner,wisdom,wish,woh,woman,women,wonder,wonderful,word,work,workout,world,worry,worse,worst,wow,write,wrong,wtf,xx,xxx,ya,yay,yeah,year,years,yes,yesterday,yo,yoga,york,young,youth,youtube,yr,yrs,yummy
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

**2. Создайте мешок слов с помощью sklearn.feature_extraction.text.TfidfVectorizer.fit_transform(). Применим его к 'tweet_stemmed' и 'tweet_lemmatized' отдельно.**
<li>Игнорируем слова, частота которых в документе строго превышает порог 0.9 с помощью max_df</li>
<li>Ограничим количество слов, попадающий в мешок, с помощью max_features = 1000</li>
<li>Исключим стоп-слова с помощью stop_words='english'</li>
<li>Отобразим Bag-of-Words модель как DataFrame. columns необходимо извлечь с помощью TfidfVectorizer.get_feature_names()</li>

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_df = 0.9, max_features = 1000, stop_words='english')

#### TfidfVectorizer - clean_tweet_stemmed

In [13]:
# Создаем the Bag-of-Words модель колонки clean_tweet_stemmed
bag_of_words = tfidf_vectorizer.fit_transform(clean_tweet_stemmed)

# Отобразим Bag-of-Words модель как DataFrame
feature_names = tfidf_vectorizer.get_feature_names()
TfidfVectorizer_cts = pd.DataFrame(bag_of_words.toarray(), columns = feature_names)
TfidfVectorizer_cts.head(5)

,abl,absolut,accept,account,act,action,actor,actual,ad,adapt,add,adventur,affirm,afternoon,age,ago,agre,ahead,aist,album,aliv,allahsoil,allow,alon,alreadi,altwaystoh,alway,amaz,america,american,amp,angel,anger,angri,anim,anniversari,announc,anoth,answer,anti,anxieti,anymor,anyon,anyth,app,appl,appreci,arriv,ask,ass,attack,august,avail,award,away,awesom,aww,babi,bad,bag,balanc,ball,bar,bc,bday,beach,bear,beat,beauti,becom,bed,beer,begin,believ,benefit,best,bestfriend,besti,better,big,bigot,bihday,bike,bing,bird,bit,bitch,black,blame,bless,blm,block,blog,blogger,blond,blue,blur,board,bodi,bong,book,bore,bought,box,boy,boyfriend,brand,break,breakfast,brexit,bride,bring,bro,broken,brother,brown,buffalo,build,bull,busi,buy,cake,calm,came,camp,campaign,cantwait,car,card,care,case,cat,caus,celebr,challeng,chanc,chang,chase,check,cheer,child,children,chill,choic,choos,christian,christina,christma,church,citi,claim,class,clean,client,climb,close,cloth,club,coach,coffe,cold,collect,colleg,color,colour,come,comment,commun,compani,complet,conc,confer,confus,congrat,congratul,connect,content,continu,control,cook,cool,count,countdown,countri,coupl,cours,cover,crazi,creat,creativ,cri,cultur,cup,current,custom,cut,cute,da,dad,daddi,daili,damn,danc,dark,date,daughter,day,dead,deal,dear,death,decid,deep,definit,delet,deletetweet,depress,deserv,design,despit,develop,die,diet,differ,dinner,direct,disappoint,disgust,disney,dj,dog,domin,donald,dont,draw,dream,dress,drink,drive,drop,dude,eah,earli,easi,eat,edit,educ,elect,els,em,...,seen,self,selfi,sell,send,sens,seri,servic,session,set,sex,sexi,shall,shame,share,shi,shit,sho,shock,shoe,shoot,shop,shot,sick,sign,silver,simpl,simul,sinc,sing,singl,sister,sit,sjw,sky,sleep,slut,small,smh,smile,snapchat,snapshot,social,someon,someth,sometim,son,song,soon,sorri,soul,sound,sourc,space,speak,special,speech,spend,spo,spring,st,sta,staff,stage,stand,star,state,stay,step,stomp,stop,store,stori,stream,street,stress,strong,stuck,student,studi,studio,stuff,stupid,style,success,suck,suicid,summer,sun,sunday,sunni,sunset,sunshin,super,suppo,suppoer,sure,surpris,surviv,sweet,swim,ta,tag,talent,talk,target,task,tattoo,tbt,tea,teach,teacher,team,tear,tech,teen,tell,term,terribl,terror,terrorist,test,tgif,th,thank,thankyou,thing,think,thought,thursday,ticket,till,time,tip,tire,today,togeth,told,tomorrow,tonight,took,total,touch,tour,town,tragedi,tragic,train,travel,treat,trend,tri,trip,true,truli,trump,trust,truth,tuesday,tune,turn,tv,tweet,twitter,type,udtapunjab,uk,understand,unit,univers,updat,upset,ur,usa,usd,use,vacat,vast,vega,vegan,vibe,vicin,victim,video,view,vine,violenc,visit,voic,vote,vs,wait,wake,walk,wan,want,war,wast,watch,water,way,weak,wear,weather,web,websit,wed,wednesday,week,weekend,welcom,went,wet,whatev,white,wife,wild,win,wine,winner,wish,woh,woman,women,wonder,word,work,workout,world,worri,worst,wow,write,wrong,wtf,xx,xxx,ya,yay,ye,yeah,year,yesterday,yo,yoga,york,young,youtub,yr,yummi
0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.

#### TfidfVectorizer - clean_tweet_lemmatized

In [14]:
# Создаем the Bag-of-Words модель колонки clean_tweet_stemmed
bag_of_words = tfidf_vectorizer.fit_transform(clean_tweet_lemmatized)

# Отобразим Bag-of-Words модель как DataFrame
feature_names = tfidf_vectorizer.get_feature_names()
TfidfVectorizer_ctl = pd.DataFrame(bag_of_words.toarray(), columns = feature_names)
TfidfVectorizer_ctl.head(5)

,able,absolutely,accept,account,act,action,actor,actually,adapt,add,adventure,affirmation,affirmations,afternoon,age,ago,agree,ahead,aist,album,alive,allahsoil,alligator,allow,altwaystoheal,amaze,america,american,americans,amp,anger,angry,animals,anniversary,announce,answer,anti,anxiety,anymore,app,apple,appreciate,arrive,ask,ass,attack,august,available,award,away,awesome,aww,baby,bad,bag,balance,ball,bank,bar,bc,bday,beach,bear,beat,beautiful,beauty,bed,beer,begin,behappy,believe,benefit,best,better,big,bihday,bike,bing,bird,bitch,bite,black,blame,bless,blm,block,blog,blogger,blonde,blue,blur,board,body,bong,book,bore,box,boy,boyfriend,boys,brand,break,breakfast,brexit,bride,bring,bro,broker,brother,brown,buffalo,build,bull,business,busy,buy,cake,calm,camp,campaign,cantwait,car,card,care,case,cat,catch,cause,celebrate,celebration,challenge,chance,change,chase,check,cheer,child,children,chill,chocolate,choice,choose,christina,christmas,church,city,claim,class,clean,click,climb,close,clothe,club,coach,cock,coffee,cold,college,color,come,comedy,comment,community,company,complete,conce,conference,confuse,congrats,congratulations,content,continue,control,cook,cool,count,countdown,country,couple,course,cover,crazy,create,creative,culture,cup,customer,customers,cut,cute,da,dad,daddy,dads,daily,damn,dance,dark,date,daughter,day,days,dead,deal,dear,death,decide,deep,delete,deletetweets,delicious,depress,depression,deserve,design,despite,development,die,diet,different,dinner,direct,disappoint,disgust,disney,dj,dog,dominate,donald,dont,download,draw,dream,dress,drink,drive,drop,eah,early,easy,eat,education,election,em,email,emotional,emotions,end,energy,...,shame,share,shit,sho,shock,shoe,shoot,shop,shy,sick,sign,silver,simple,simulation,simulator,sing,single,sister,sit,sjw,sky,sleep,slut,small,smh,smile,snapchat,snapshot,social,son,song,songs,soon,sorry,soul,sound,source,south,space,spain,speak,special,spend,spos,spring,squad,st,sta,staed,staff,stage,staing,stand,star,stas,state,stay,step,stick,stomp,stop,store,stories,story,stream,street,stress,strong,students,studio,study,stuff,stupid,style,success,successful,suck,summer,sun,sunday,sunny,sunset,sunshine,super,suppo,suppoers,sure,surprise,survive,sweet,swim,ta,tag,talk,tampa,target,task,tattoo,tbt,tea,teach,team,tear,tech,teen,tell,term,terrorism,test,texas,tgif,th,thank,thankful,thankyou,thing,things,think,tho,thoughts,throw,thursday,ticket,till,time,tip,tire,today,tomorrow,tonight,totally,touch,tour,town,track,tragedy,tragic,train,travel,treat,tree,trend,trip,true,truly,trump,trust,truth,try,tuesday,tune,turn,tv,tweet,twitter,type,udtapunjab,uk,understand,unite,university,update,upset,ur,usa,usd,use,useful,vacation,vast,vegan,vegas,vibes,vicinity,victims,video,videos,view,vine,violence,visit,voice,vote,vs,wait,wake,wakeup,walk,wan,want,war,waste,watch,water,way,ways,weak,wear,weather,web,website,wed,wednesday,week,weekend,weeks,welcome,wet,white,wife,wild,win,wine,winner,wisdom,wish,woh,woman,women,wonder,wonderful,word,work,workout,world,worry,worse,worst,wow,write,wrong,wtf,xx,xxx,ya,yay,yeah,year,years,yes,yesterday,yo,yoga,york,young,youth,youtube,yr,yrs,yummy
0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,

**3. Создайте мешок слов с помощью sklearn.feature_extraction.text.HashingVectorizer.fit_transform(). Применим его к 'tweet_stemmed' и 'tweet_lemmatized' отдельно.**
<li>Игнорируем слова, частота которых в документе строго превышает порог 0.9 с помощью max_df</li>
<li>Ограничим количество слов, попадающий в мешок, с помощью max_features = 1000.(можно изменить)</li>
<li>Исключим стоп-слова с помощью stop_words='english'</li>

**4. Проверьте ваши векторайзеры на корпусе который использовали на вебинаре, составьте таблицу метод векторизации и скор который вы получили (в методах векторизации по изменяйте параметры что бы добиться лучшего скора) обратите внимание как падает/растёт скор при уменьшении количества фичей, и изменении параметров, так же попробуйте применить к векторайзерам PCA для сокращения размерности посмотрите на качество сделайте выводы**